In [ ]:
# 包导入
import tensorflow as tf 
from tensorflow import keras
import numpy as np
import random

In [ ]:
# 数据处理
# 验证小区上的流量，标签为小区下一个小时流量
# y_true为小区下个小时的流量
# 数据集为[24,4]的张量，将([i,3]+[i,4])=y_train[]
# 数据为：平均人数，最大人数，上行流量，下行流量，总流量（标签）
# 用于训练时总流量要向后移动一小时
# #


# 数据生成的参数设置
# #
hour_rate = [0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 
             0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1, 0.2, 0.3, 0.1,0.2, 0.3]
people = 1000
down_flow = 5000.0
up_flow = 100.0
file_path = "data.tfrecord"


# 写数据到TFrecord文件
# #
# 生成首个数据，接下来的数据在前23小时上加一小时
first_day = np.zeros(120)
all_day = np.zeros(120000)
for i in range(0, 24):
    avg_people = hour_rate[i%24] * random.randint(30,80) * people / 10 
    max_people = avg_people * random.randint(20, 40) / 10
    down_f = hour_rate[i%24] * random.randint(30,80) * down_flow / 10
    up_f = hour_rate[i%24] * random.randint(30, 80) *  up_flow / 10
    first_day[i*5] = avg_people
    first_day[i*5+1] = max_people
    first_day[i*5+2] = up_f
    first_day[i*5+3] = down_f
    first_day[i*5+4] = up_f + down_f

# 储存第一组数据到总数据数组
for i in range(0, 120):
    all_day[i] = first_day[i] 

pre_day = first_day
for j in range(1, 1000):
    next_day = np.zeros(120)
    for m in range(0, 115):
        next_day[m] = pre_day[m+5]
        #输入新加的5个数据
        avg_people = hour_rate[j%24] * random.randint(30,80) * people / 10 
        max_people = avg_people * random.randint(20, 40) / 10
        down_f = hour_rate[j%24] * random.randint(30,80) * down_flow / 10
        up_f = hour_rate[j%24] * random.randint(30, 80) *  up_flow / 10
        next_day[115] = avg_people
        next_day[116] = max_people
        next_day[117] = up_f
        next_day[118] = down_f
        next_day[119] = up_f + down_f
    for n in range(0, 120):
        all_day[j*120+n] = next_day[n]
    pre_day = next_day
# 写入数据
with tf.io.TFRecordWriter(file_path) as files:
    # 第一个数据的写入
    example_first = tf.train.Example(
        features=tf.train.Features(
            feature={
                "data": tf.train.Feature(
                    float_list=tf.train.FloatList(value=all_day))
                }))
    files.write(example_first.SerializePartialToString())
    


In [ ]:
# 读数据函数,返回解析后的结果
def read_datas(filepath):
    feature_description = {
        "data": tf.io.VarLenFeature(dtype=float)
    }

    file_dataset = tf.data.TFRecordDataset(filepath).batch(5)

    for serialized_example in file_dataset:
        parsed_example = tf.io.parse_example(serialized_example,                                                                                                       feature_description)
    
    return parsed_example

In [ ]:
# 读取数据并简单处理
datas = read_datas(file_path)["data"].values
reshape_datas = tf.reshape(datas, [24000,5])
x_data = reshape_datas[0:24000,0:4]
y_data = reshape_datas[0:24000,4]
print(reshape_datas)

In [70]:
_data = np.array(x_data)
data_4 = np.zeros(shape=[1,24,4,1])
for i in range(0, 24):
    for j in range(0, 4):
          data_4[0,i,j,0] = _data[i,j]

train_4 = tf.constant(data_4, dtype=float)
print(train_4)

tf.Tensor(
[[[[ 670.]
   [2345.]
   [  47.]
   [3900.]]

  [[1080.]
   [4320.]
   [ 128.]
   [5800.]]

  [[1440.]
   [5040.]
   [ 117.]
   [9000.]]

  [[ 780.]
   [2028.]
   [  41.]
   [3100.]]

  [[ 860.]
   [2752.]
   [ 138.]
   [4300.]]

  [[1800.]
   [4500.]
   [ 114.]
   [7800.]]

  [[ 640.]
   [1664.]
   [  45.]
   [3600.]]

  [[ 840.]
   [1764.]
   [ 108.]
   [8000.]]

  [[1290.]
   [3096.]
   [ 177.]
   [8250.]]

  [[ 670.]
   [1407.]
   [  40.]
   [2350.]]

  [[ 720.]
   [2376.]
   [ 116.]
   [6400.]]

  [[1950.]
   [5850.]
   [ 114.]
   [8250.]]

  [[ 500.]
   [1950.]
   [  33.]
   [1650.]]

  [[1100.]
   [2640.]
   [  96.]
   [3200.]]

  [[1500.]
   [4500.]
   [ 138.]
   [5700.]]

  [[ 490.]
   [1274.]
   [  57.]
   [2000.]]

  [[ 740.]
   [2960.]
   [  94.]
   [5100.]]

  [[1890.]
   [5292.]
   [ 105.]
   [7050.]]

  [[ 370.]
   [ 814.]
   [  53.]
   [1850.]]

  [[1420.]
   [2982.]
   [ 102.]
   [6800.]]

  [[ 930.]
   [2790.]
   [ 144.]
   [9900.]]

  [[ 500.]
   [1850.]
 

In [ ]:
# x_data数据展示
print(x_data)

In [ ]:
# y_data数据展示
print(y_data[24])

In [ ]:
# y_all是每个数据张量的下一个数据张量的第一行第五列
y_all_data = np.zeros(1000)
for i in range(0, 1000):
    y_all_data[i] = y_data[i*24+1]
y_all = tf.constant(y_all_data, dtype=float)
print(y_all) 

In [ ]:
# 训练数据，验证数据，测试数据的构建
# 比例为0.7：0.2：0.1

y_train = y_all[0:700]
X_train = x_data[0:16800,0:4]

y_evaluate = y_all[700:900]
X_evaluate = x_data[16800:21600,0:4]

y_test = y_all[900:1000]
X_test = x_data[21600:24000,0:4]

In [ ]:
#############################################模型构建####################################################################

In [ ]:
# CNN模块创建
# 步频为1，输入输出的空间维度不变[24,4]
udn_model = keras.models.Sequential([
    keras.layers.Conv2D(128, 2, activation="relu", padding="same",
                         input_shape=[24, 4, 1]),
    keras.layers.Conv2D(64, 2, activation="relu", padding="same"),
    keras.layers.Conv2D(32, 2, activation="relu", padding="same"),
    keras.layers.Conv2D(32, 2, activation="relu", padding="same"),
    keras.layers.MaxPool2D(2),
    keras.layers.Conv2D(16, 2, activation="relu", padding="same"),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation="relu"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation="relu")
])

udn_model.summary()

In [73]:
y = y_all[10]
y_4 = tf.constant([1.0])
print(y)
print(y_4)

tf.Tensor(8364.0, shape=(), dtype=float32)
tf.Tensor([1.], shape=(1,), dtype=float32)


In [72]:
model = keras.models.Sequential([
    keras.layers.Conv2D(128, 2, activation="relu", padding="same",
                        input_shape=[24,4,1])
])

SGD = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
model.compile(
    loss=negative_log_likelihood,
    optimizer=SGD
)

y = model.fit(
    x=train_4,
    y=y_4,
)


Train on 1 samples
1/1 [==============================] - 1s 1s/sample - loss: nan


In [ ]:
# 损失函数为负对数似然（negative log likelihood）
def negative_log_likelihood(y_true, y_pred, eps=1e-15):
    p = keras.backend.clip(y_pred, eps, 1-eps)
    loss = keras.backend.sum(-y_true * keras.backend.log(p) - (1-y_true) * keras.backend.log(1-p))
    return keras.backend.mean(loss)

In [ ]:
# 编译模块
SGD = keras.optimizers.SGD(lr=0.2, momentum=0.9, decay=0.01)
udn_model.compile(
    loss=negative_log_likelihood,
    optimizer=SGD
)

In [ ]:
input_shape = (4, 28, 28, 3)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv2D(128, 2, activation='relu', padding="same",input_shape=[24, 4, 1])(train_4)
print(y.shape)

In [ ]:
# 训练模块
# 在训练时运用mini-batch方法进行小批量梯度、
# 对每个小时进行预测
udn_model.fit(
    x=train_4,
    y=y_all[0],
    callbacks="EarlyStopping"
)

In [ ]:
# 验证模块
udn_model.evaluate(

)